In [ ]:
%pip install geopandas plotly

In [ ]:
import re
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

# **Variables**

In [ ]:
PROJECT_NAME = "AreaVerde"

# **Load Data**

In [ ]:
# Load data
project = dh.get_or_create_project(PROJECT_NAME)

spira = project.get_dataitem("spira_flow_data_2024")
spira5m = project.get_dataitem("spire_flow5m_2024")
accuracy = project.get_dataitem("spira_accur_data_2024")
accuracy5m = project.get_dataitem("spire_flow5m_accur_2024")

In [ ]:
spira = spira.as_df()

In [ ]:
spira_id_map = spira[['spira_code', 'spira_unique_id']].drop_duplicates()

In [ ]:
spira_locations = spira[['spira_unique_id', 'spira_code', 'longitudine', 'latitudine']].drop_duplicates()

In [ ]:
spira_id_map_sel = spira_id_map[spira_id_map['spira_unique_id'].between(200, 250)]
spira_id_map_sel

In [ ]:
sel = [[('sensor_id', '==', id)] for id in spira_id_map_sel['spira_code']]

In [ ]:
start_date = datetime(2024, 6, 1)
end_date = datetime(2024, 7, 31)

In [ ]:
spira5m.download("./tmp/Spira5mFlowData.parquet", overwrite=True)
spira5m = pd.read_parquet("./tmp/Spira5mFlowData.parquet", filters=sel, engine="pyarrow")
spira5m = pd.merge(spira5m, spira_id_map, left_on='sensor_id', right_on='spira_code', how='left')
spira5m = spira5m[['spira_unique_id', 'spira_code', 'value', 'start']].rename(columns={'value': 'count', 'start': 'DateTime'})
spira5m['DateTime'] = pd.to_datetime(spira5m['DateTime'], format='%Y-%m-%d %H:%M')
spira5m = spira5m[spira5m['DateTime'].between(start_date, end_date, inclusive='left')]
spira5m = spira5m.sort_values('DateTime')
spira5m

In [ ]:
accuracy = accuracy.as_df()
accuracy['DateTime'] = pd.to_datetime(accuracy['DateTime'])
accuracy = accuracy[accuracy['DateTime'].between(start_date, end_date, inclusive='left')]
accuracy = accuracy.sort_values('DateTime')
accuracy

# **Plot spira data**

In [ ]:
def plot_spira(spira_id, start=0, days=0):
    spira5m_sel = spira5m[spira5m['spira_code'] == spira_id]
    accuracy_sel = accuracy[accuracy['spira_code'] == spira_id]
    plt.figure(figsize=(12,3))
    plt.plot(spira5m_sel.iloc[24*12*start:24*12*(start+days) if days else -1]['DateTime'], 
             spira5m_sel.iloc[24*12*start:24*12*(start+days) if days else -1]['count'])
    plt.plot(accuracy_sel.iloc[24*start:24*(start+days) if days else -1]['DateTime'], 
             accuracy_sel.iloc[24*start:24*(start+days) if days else -1]['count'])
    
def map_spira(spira_id):
    data_points = spira_locations[spira_locations['spira_code'] == spira_id].copy()
    data_points['size'] = 1
    fig = px.scatter_mapbox(
        data_points,
        lat='latitudine',
        lon='longitudine',
        mapbox_style='open-street-map',
        size='size',
        size_max=12,
        zoom=11.75,
        height=400,
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
map_spira('0.127 3.87 2 1')
plot_spira('0.127 3.87 2 1')
plot_spira('0.127 3.87 2 1', start=37, days=12)
plot_spira('0.127 3.87 2 1', start=47, days=1)
plot_spira('0.127 3.87 2 1', start=10, days=1)

In [ ]:
map_spira('0.127 4.33 8 1')
plot_spira('0.127 4.33 8 1')

In [ ]:
map_spira('0.127 3.88 10 1')
plot_spira('0.127 3.88 10 1')

In [ ]:
map_spira('0.127 4.27 6 1')
plot_spira('0.127 4.27 6 1')